# "Tardigrades: from genestealers to space marines"

Project #4

*Lab Journal by Artem Vasilev and Tatiana Lisitsa*

---

# **Preparing**

Update packages:

`$ sudo apt update && sudo apt upgrade`

Create virtual environment from `environment.yaml` file:

`$ mamba env create -f environment.yaml -p /home/<user>/<anaconda3 or conda>/envs/<env_name>`

Specify your username, an install path and name of environment

Activate it:

`$ mamba activate <env_name>`

---

# **Data processing**

Many of the steps are done using **SnakeMake**. For details see `Snakefile`'s contents

## <u> Obtaining data. Genome sequence

### Download assembled genome

`$ snakemake --cores=all -p data/assembled_genome.fna.gz`

### Download precomputed AUGUSTUS results

`$ snakemake --cores=all -p data/augustus_whole.fasta`

After running the commands above, your repository will have the following structure:
```
-/Practice/Project_4/
 |- data
       |- assembled_genome.fna.gz (17.6 MB)
       |- augustus_whole.fasta (7.3 MB)
       |- augustus_whole.gff (27.5 mB)
       |- peptides.fa (737 bytes)
```

## <u> Structural annotation

### Parsing .gff file

```
$ mkdir results
$ python3 parse_gff.py
Enter the path to the .gff file: data/augustus_whole.gff
Do you want to specify the name of output file? (y/n): y
Enter the name of output file: results/augustus_whole_parsed.fasta
Your job is done!
```

`$ cat results/augustus_whole_parsed.fasta | grep '>' | wc -l`  # 16435

After running the commands above, your repository will have the following structure:
```
-/Practice/Project_4/
 |- data
       ...
 |- results
       |- augustus_whole_parsed.fasta (7.2 MB)
```

### Download a list of peptides that were associated with the DNA

Link: https://disk.yandex.ru/d/xJqQMGX77Xueqg

It is already in the `/data` folder

## <u> Physical localization

### Local alignment-based search

#### Classic BLAST+

`$ snakemake --cores=all -p results/blast/local_database.pdb`

`$ snakemake --cores=all -p results/blast/aligned_blast`

After running the commands above, your repository will have the following structure:
```
-/Practice/Project_4/
 |- data
       ...
 |- results
       |- blast
            |- aligned_blast (3.0 kB)
            |- 8 items local_database (8.6 MB)
```

#### `-outfmt args`

`-outfmt 0`: Pairwise, tab-delimited format.

`-outfmt 5`: XML format.

`-outfmt 7`: Text format with comment lines.

`-outfmt 8`: Tabular format with column headers.

`-outfmt 9`: Archive (ASN.1) format.

`-outfmt 11`: Search strategy output.

Query ID (`qseqid`): Identifier of the query sequence.

Subject ID (`sseqid`): Identifier of the subject (database) sequence.

Percentage of identical matches (`pident`): Percentage of identical matches between the query and subject.

Alignment length (`length`): Length of the alignment between the query and subject.

Number of mismatches (`mismatch`): Number of mismatches between the query and subject.

Number of gap openings (`gaps`): Number of gap openings in the alignment.

Start of alignment in query (`qstart`): Start position of the alignment in the query sequence.

End of alignment in query (`qend`): End position of the alignment in the query sequence.

Start of alignment in subject (`sstart`): Start position of the alignment in the subject sequence.

End of alignment in subject (`send`): End position of the alignment in the subject sequence.

Expect value (`evalue`): Expect value of the alignment.

Bit score (`bitscore`): Bit score of the alignment.

### Extract proteins of interest from the initial file

`$ snakemake --cores=all -p results/extracted_proteins.fasta`

After running the command above, your repository will have the following structure:
```
-/Practice/Project_4/
 |- data
       ...
 |- results
       |- blast
            ...
       ...
       |- augustus_whole_parsed.fasta.fai (429.7 kB)
       |- extracted_proteins.fasta (16.0 kB)
       |- unique_proteins.txt (218 bytes)
```

## <u> Localization prediction

### [WoLF PSORT](https://wolfpsort.hgc.jp/)

Parameters:

1. Please select an organism type:
  - Animal
2. Please select input method:
  - From File
3. Input Filename:
  - `extracted_proteins.fasta`
4. Submit
5. Copy results from the 1st `g*` (the example of the results is shown below)

Output information:

- nucl: Nucleus
- cyto_nucl: Cytoplasmic and Nuclear
- cyto: Cytoplasm
- E.R.: Endoplasmic Reticulum
- golg: Golgi Apparatus
- extr: Extracellular (indicates the protein is predicted to be located outside the cell)
- plas: Plasma membrane (the membrane surrounding the cell)
- mito: Mitochondrion (the membrane-bound organelle responsible for cellular respiration)
- lyso: Lysosome (the membrane-bound organelle containing digestive enzymes)

#### Parsing WoLF results

Command line version (not convenient enough because of output format)

```
$ python3 parse_wolf.py
Enter the results of WoLF PSORT (press Enter, if you are done):
...
Do you want to specify the name of output file? (y/n): y
Enter the name of output file without extension: results/wolf
Your job is done!
```

In [84]:
results_wolf = '''g10513 details nucl: 20, cyto_nucl: 14.5, cyto: 7, extr: 3, E.R.: 1, golg: 1
g10514 details nucl: 19, cyto_nucl: 15, cyto: 9, extr: 3, mito: 1
g11320 details plas: 24.5, extr_plas: 16, extr: 6.5, lyso: 1
g11513 details cyto: 17, cyto_nucl: 12.8333, cyto_mito: 9.83333, nucl: 7.5, E.R.: 3, mito: 1.5, plas: 1, pero: 1, golg: 1
g11806 details nucl: 18, cyto_nucl: 11.8333, mito: 5, extr: 4, cyto: 3.5, cyto_pero: 2.66667, cysk_plas: 1
g11960 details nucl: 32
g12388 details extr: 25, plas: 4, mito: 2, lyso: 1
g12510 details plas: 29, cyto: 3
g12562 details extr: 30, lyso: 2
g1285 details extr: 25, plas: 5, mito: 1, lyso: 1
g13530 details extr: 13, nucl: 6.5, lyso: 5, cyto_nucl: 4.5, plas: 3, E.R.: 3, cyto: 1.5
g14472 details nucl: 28, plas: 2, cyto: 1, cysk: 1
g15153 details extr: 32
g15484 details nucl: 17.5, cyto_nucl: 15.3333, cyto: 12, cyto_mito: 6.83333, plas: 1, golg: 1
g16318 details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1
g16368 details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1
g2203 details plas: 29, nucl: 2, golg: 1
g3428 details mito: 18, cyto: 11, extr: 2, nucl: 1
g3679 details extr: 26, mito: 2, lyso: 2, plas: 1, E.R.: 1
g4106 details E.R.: 14.5, E.R._golg: 9.5, extr: 7, golg: 3.5, lyso: 3, pero: 2, plas: 1, mito: 1
g4970 details plas: 32
g5237 details plas: 24, mito: 8
g5443 details extr: 28, nucl: 3, cyto: 1
g5467 details extr: 27, plas: 4, mito: 1
g5502 details extr: 31, lyso: 1
g5503 details extr: 29, plas: 1, mito: 1, lyso: 1
g5510 details plas: 23, mito: 7, E.R.: 1, golg: 1
g5616 details extr: 31, mito: 1
g5641 details extr: 31, lyso: 1
g5927 details nucl: 30.5, cyto_nucl: 16.5, cyto: 1.5
g702 details extr: 29, plas: 2, lyso: 1
g7861 details nucl: 16, cyto_nucl: 14, cyto: 8, plas: 5, pero: 1, cysk: 1, golg: 1
g8100 details nucl: 16.5, cyto_nucl: 12.5, cyto: 7.5, plas: 5, extr: 2, E.R.: 1
g8312 details nucl: 15.5, cyto_nucl: 15.5, cyto: 12.5, mito: 2, plas: 1, golg: 1'''

In [142]:
import re
import pandas as pd
import numpy as np
from IPython.display import display


results_wolf_list = ' '.join(results_wolf.split('\n'))
results_list = results_wolf_list.split(' ')
results_dict = {}

for word in results_list:
  if re.match(r'g\d+', word):
    id = word
    results_dict[id] = {}
  elif word != 'details':
    if word.find(':') != -1:
      pos = word.strip(':')
    else:
      results_dict[id][pos] = word.strip(',')

# This block is an alternative version of text format output
# for id, pos in results_dict.items():
#   print(f'{id}:')
#   for name, value in pos.items():
#     print(f'\t{name}: {value}')

In [145]:
column_names = ['id',
                'nucl',
                'cyto',
                'cyto_nucl',
                'mito',
                'cyto_mito',
                'pero',
                'cyto_pero',
                'extr',
                'plas',
                'extr_plas',
                'cysk',
                'cysk_plas',
                'E.R.',
                'golg',
                'E.R._golg',
                'lyso']

n_rows = len(list(results_dict.keys()))
df_wolf = pd.DataFrame(np.zeros((n_rows, len(column_names)), dtype=int))
df_wolf.columns = column_names
df_wolf.id = list(results_dict.keys())
df_wolf = df_wolf.set_index('id')

for id, pos in results_dict.items():
  for name, value in pos.items():
    df_wolf.loc[id, name] = value

display(df_wolf)

,nucl,cyto,cyto_nucl,mito,cyto_mito,pero,cyto_pero,extr,plas,extr_plas,cysk,cysk_plas,E.R.,golg,E.R._golg,lyso
id,,,,,,,,,,,,,,,,
g10513,20,7,14.5,0,0,0,0,3,0,0,0,0,1,1,0,0
g10514,19,9,15,1,0,0,0,3,0,0,0,0,0,0,0,0
g11320,0,0,0,0,0,0,0,6.5,24.5,16,0,0,0,0,0,1
g11513,7.5,17,12.8333,1.5,9.83333,1,0,0,1,0,0,0,3,1,0,0
g11806,18,3.5,11.8333,5,0,0,2.66667,4,0,0,0,1,0,0,0,0
g11960,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
g12388,0,0,0,2,0,0,0,25,4,0,0,0,0,0,0,1
g12510,0,3,0,0,0,0,0,0,29,0,0,0,0,0,0,0
g12562,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,2


### [TargetP Server](https://services.healthtech.dtu.dk/service.php?TargetP-2.0)


Parameters:

1. Organism group:
  - Non-plant
2. Output format:
  - Long output
3. Format directly from your local disk:
  - `extracted_proteins.fasta`
4. Submit
5. Downloads > Prediction summary
6. `$ mv ~/Downloads/output_protein_type.txt results/`

After running the steps above, your repository will have the following structure:
```
-/Practice/Project_4/
 |- data
       ...
 |- results
       |- blast
            ...
       ...
       |- output_protein_type.txt (1.9 kB)
```

In [147]:
df_target = pd.read_csv('/output_protein_type.txt', sep='\t', skiprows=1)
df_target = df_target.set_index('# ID')

merged_df = pd.merge(df_wolf, df_target, how='left', left_index=True, right_index=True)
display(merged_df)

,nucl,cyto,cyto_nucl,mito,cyto_mito,pero,cyto_pero,extr,plas,extr_plas,...,cysk_plas,E.R.,golg,E.R._golg,lyso,Prediction,OTHER,SP,mTP,CS Position
id,,,,,,,,,,,,,,,,,,,,,
g10513,20,7,14.5,0,0,0,0,3,0,0,...,0,1,1,0,0,OTHER,0.999999,0.000001,0.000000,NaN
g10514,19,9,15,1,0,0,0,3,0,0,...,0,0,0,0,0,OTHER,0.999543,0.000349,0.000107,NaN
g11320,0,0,0,0,0,0,0,6.5,24.5,16,...,0,0,0,0,1,SP,0.000184,0.999816,0.000000,CS pos: 20-21. AYS-AG. Pr: 0.7236
g11513,7.5,17,12.8333,1.5,9.83333,1,0,0,1,0,...,0,3,1,0,0,OTHER,0.999434,0.000401,0.000164,NaN
g11806,18,3.5,11.8333,5,0,0,2.66667,4,0,0,...,1,0,0,0,0,OTHER,0.998977,0.000887,0.000136,NaN
g11960,32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,OTHER,0.999996,0.000002,0.000002,NaN
g12388,0,0,0,2,0,0,0,25,4,0,...,0,0,0,0,1,SP,0.000490,0.999481,0.000029,CS pos: 16-17. ASA-SS. Pr: 0.6485
g12510,0,3,0,0,0,0,0,0,29,0,...,0,0,0,0,0,OTHER,0.999738,0.000099,0.000163,NaN
g12562,0,0,0,0,0,0,0,30,0,0,...,0,0,0,0,2,SP,0.000076,0.999923,0.000001,CS pos: 16-17. SYA-AN. Pr: 0.7910


In [148]:
nucl_df = merged_df[['nucl', 'cyto_nucl', 'Prediction']]
display(nucl_df)

,nucl,cyto_nucl,Prediction
id,,,
g10513,20,14.5,OTHER
g10514,19,15,OTHER
g11320,0,0,SP
g11513,7.5,12.8333,OTHER
g11806,18,11.8333,OTHER
g11960,32,0,OTHER
g12388,0,0,SP
g12510,0,0,OTHER
g12562,0,0,SP


In [168]:
nucl_df_copy = nucl_df.copy()
nucl_df_copy['nucl'] = pd.to_numeric(nucl_df_copy['nucl'], errors='coerce')
nucl_df_copy['cyto_nucl'] = pd.to_numeric(nucl_df_copy['cyto_nucl'], errors='coerce')
filtered_df = nucl_df_copy.query('nucl > 5 and cyto_nucl >= 0 and Prediction == "OTHER"')

proteins_id = filtered_df.index.to_list()
for id in proteins_id:
  print(id)

g10513
g10514
g11513
g11806
g11960
g14472
g15484
g16318
g16368
g5927
g7861
g8100
g8312


Copy this id's into new .txt file:

`$ touch results/filtered_proteins.txt`

`$ nano results/filtered_proteins.txt`

`Ctrl-Shift-V, Ctrl-X, Y, Enter`

After running the steps above, your repository will have the following structure:
```
-/Practice/Project_4/
 |- data
       ...
 |- results
       |- blast
            ...
       ...
       |- filtered_proteins.txt (87 bytes)
```

### Parsing filtered protein sequences

`$ snakemake --cores=all -p results/extracted_filtered_proteins.fasta`

After running the steps above, your repository will have the following structure:
```
-/Practice/Project_4/
 |- data
       ...
 |- results
       |- blast
            ...
       ...
       |- extracted_filtered_proteins.fasta (8.6 kB)
```

---

# **Data analyzing**

## <u> BLAST search

1. Go to the [BLAST](https://blast.ncbi.nlm.nih.gov/) page
2. Select **Protein BLAST**
3. Copy sequences from `extracted_filtered_proteins.fasta` <u> one by one </u>
4. Change parameters:
  - Database: UniProtKB/Swiss-Prot(swissprot)
5. BLAST

No significant similarity found for these id's:

- g10513
- g10514
- g11806
- g16318
- g16368

## <u> Pfam prediction

### [HMMER](https://www.ebi.ac.uk/Tools/hmmer/)

1. Select "Search"
2. Select the suitable tool (in our case it is **hmmscan**)
3. Select the **Pfam** database
4. Copy sequences from `extracted_filtered_proteins.fasta` <u> one by one </u>
5. Submit

No hits were found for query for these id's:

- g10513
- g10514
- g11806
- g14472
- g16318
- g16368
- g5927